In [2]:
!pip install sentencepiece 

!pip install smart_open 
!pip install gensim
!pip install numpy 

!pip install tqdm 

!pip install pymorphy2

     |████████████████████████████████| 1.1MB 9.1MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 8.2MB 8.2MB/s 


Обработка данных

In [3]:
import re
import sentencepiece as spm

import smart_open as sm
import gensim
import numpy as np

from tqdm.notebook import tqdm

import pymorphy2

In [4]:
def read_data(path):
    with open(path, "r") as f:
        for line in f:
            cat, headline, text = line.strip().split("\t")
            yield cat, headline, text

def tokenize_text(text):
    text = text.lower()
    words = re.findall(r'\b\w+\b', text.lower())
    return words

def prepare_data(path):
    data = []
    for cat, headline, text in read_data(path):
        item = {}
        item["cat"] = cat
        item["headline"] = tokenize_text(headline)
        item["text"] = [tokenize_text(sentance) for sentance in re.split(r"[.!?]", text) if len(sentance) > 20]
        data.append(item)
    return data

In [5]:
data_train = prepare_data("news_train.txt")
data_test = prepare_data("news_test.txt")
data_train[0]

{'cat': 'sport',
 'headline': ['овечкин',
  'пожертвовал',
  'детской',
  'хоккейной',
  'школе',
  'автомобиль'],
 'text': [['нападающий',
   'вашингтон',
   'кэпиталз',
   'александр',
   'овечкин',
   'передал',
   'детской',
   'хоккейной',
   'школе',
   'автомобиль',
   'полученный',
   'им',
   'после',
   'окончания',
   'матча',
   'всех',
   'звезд',
   'национальной',
   'хоккейной',
   'лиги',
   'нхл'],
  ['об', 'этом', 'сообщается', 'на', 'официальном', 'сайте', 'лиги'],
  ['автомобиль',
   'honda',
   'accord',
   'был',
   'подарен',
   'хоккеисту',
   'по',
   'решению',
   'спонсоров',
   'мероприятия'],
  ['игрок',
   'нхл',
   'пожертвовал',
   'машину',
   'спортивной',
   'школе',
   'nova',
   'cool',
   'cats',
   'special',
   'hockey',
   'inc'],
  ['которая', 'расположена', 'в', 'штате', 'вирджиния'],
  ['овечкин',
   'общается',
   'с',
   '10',
   'летней',
   'девочкой',
   'анной',
   'шоб',
   'с',
   'синдромом',
   'дауна',
   'которая',
   'занимается

In [6]:
def postprocess_tokens_(data):
    morph = pymorphy2.MorphAnalyzer()
    for item in tqdm(data):
        item["headline"] = [morph.parse(word)[0].normal_form for word in item["headline"]] 
        item["text"] = [[morph.parse(word)[0].normal_form for word in sentance] for sentance in item["text"]] 

postprocess_tokens_(data_train)
postprocess_tokens_(data_test)

data_train[0]

{'cat': 'sport',
 'headline': ['овечкин',
  'пожертвовать',
  'детский',
  'хоккейный',
  'школа',
  'автомобиль'],
 'text': [['нападать',
   'вашингтон',
   'кэпиталзти',
   'александр',
   'овечкин',
   'передать',
   'детский',
   'хоккейный',
   'школа',
   'автомобиль',
   'получить',
   'они',
   'после',
   'окончание',
   'матч',
   'весь',
   'звезда',
   'национальный',
   'хоккейный',
   'лига',
   'нхл'],
  ['о', 'это', 'сообщаться', 'на', 'официальный', 'сайт', 'лига'],
  ['автомобиль',
   'honda',
   'accord',
   'быть',
   'подарить',
   'хоккеист',
   'по',
   'решение',
   'спонсор',
   'мероприятие'],
  ['игрок',
   'нхл',
   'пожертвовать',
   'машина',
   'спортивный',
   'школа',
   'nova',
   'cool',
   'cats',
   'special',
   'hockey',
   'inc'],
  ['который', 'расположить', 'в', 'штат', 'вирджиния'],
  ['овечкин',
   'общаться',
   'с',
   '10',
   'летний',
   'девочка',
   'анна',
   'чтоб',
   'с',
   'синдром',
   'даун',
   'который',
   'заниматься',
   '

In [7]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    min_count=10,
    window=2,
    size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

sentences = [item["headline"] for item in data_train]
sentences.extend([sentance for item in data_train for sentance in item["text"]])

w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(48635425, 91978410)

In [8]:
print(w2v_model.wv.most_similar('октябрь'))

[('сентябрь', 0.7763291597366333), ('июль', 0.7640365362167358), ('декабрь', 0.7628675103187561), ('май', 0.7626175284385681), ('март', 0.7562806606292725), ('ноябрь', 0.7526770830154419), ('апрель', 0.7513618469238281), ('январь', 0.7410556077957153), ('август', 0.7205405235290527), ('февраль', 0.7161135673522949)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [11]:
max_item_len = 140
label2idx = {}
def prepere_tfidf(data):
    X = []
    y = []

    for item in tqdm(data):
        label = item["cat"]
        headline = item["headline"]
        text = item["text"]

        label_idx = label2idx.get(label, len(label2idx))
        label2idx[label] = label_idx

        word_idx = 0
        sent_idx = 0
        pos_in_sent = 0

        x = []

        while word_idx < max_item_len:
            if word_idx < len(headline):
                x.append(headline[word_idx])
                word_idx += 1
            else:
                if pos_in_sent < len(text[sent_idx]):
                    x.append(text[sent_idx][pos_in_sent])
                    word_idx += 1
                    pos_in_sent += 1
                elif sent_idx < len(text) - 1:
                    sent_idx += 1
                    pos_in_sent = 0
                else:
                    x.append("PLACEHOLDER")
                    word_idx += 1

        X.append(" ".join(x))
        y.append(label_idx)
    
    return X, y

X_train_idf, y_train = prepere_tfidf(data_train)
X_test_idf, y_test = prepere_tfidf(data_test)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train_idf = tfidf.fit_transform(X_train_idf)
X_test_idf = tfidf.transform(X_test_idf)

In [13]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(n_jobs=-1)
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

accuracy = 0.8593333333333333


In [16]:
from sklearn.model_selection import GridSearchCV

hyper = {'C' : [1, 4, 10, 100], 'solver': ['lbfgs']}
gd = GridSearchCV(estimator=
                LogisticRegression(multi_class='multinomial', random_state=17, n_jobs=4),
                param_grid=hyper)
gd.fit(X_train_idf, y_train)
print(gd.best_score_)
print(gd.best_estimator_)

0.8648
LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=4, penalty='l2',
                   random_state=17, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [18]:
clf_lr = gd.best_estimator_
clf_lr.fit(X_train_idf, y_train)
preds = clf_lr.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

accuracy = 0.879


In [19]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

accuracy = 0.7983333333333333
